In [4]:
import os
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

## Load Model

In [5]:
model = YOLO('clasify_model.pt')

## Implementasi Klasifikasi

In [6]:
def classify_apples(image_path):
    
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    
    output_base = f'output/{image_name}'
    output_dirs = [f'{output_base}/red', f'{output_base}/yellow', f'{output_base}/green']
    for folder in output_dirs:
        os.makedirs(folder, exist_ok=True)

    class_counters = {'red': 1, 'yellow': 1, 'green': 1}

    img = cv2.imread(image_path)

    results = model(img)

    annotated_img = img.copy()

    for result in results[0].boxes:
        # Ekstrak informasi bounding box dan label
        box = result.xyxy[0].cpu().numpy().astype(int)
        label = int(result.cls.cpu().numpy())  

        class_mapping = {0: 'green', 1: 'red', 2: 'yellow'}
        class_label = class_mapping.get(label, 'unknown')

        class_counter = class_counters[class_label]
        class_counters[class_label] += 1

        # Gambar bounding box pada gambar
        color_map = {'red': (0, 0, 255), 'yellow': (0, 255, 255), 'green': (0, 255, 0)}
        color = color_map.get(class_label, (255, 255, 255))
        cv2.rectangle(annotated_img, (box[0], box[1]), (box[2], box[3]), color, 2)
        cv2.putText(annotated_img, f"{class_label} {class_counter}", (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        crop_img = img[box[1]:box[3], box[0]:box[2]]


        output_path = f'{output_base}/{class_label}/{class_label}_{class_counter}.jpg'
        cv2.imwrite(output_path, crop_img)

    
    annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(10, 10))
    plt.imshow(annotated_img_rgb)
    plt.axis('off')
    plt.title(f'Hasil deteksi objek: {image_name}')
    plt.show()

    annotated_output_path = f'{output_base}/annotated_{image_name}.jpg'
    cv2.imwrite(annotated_output_path, annotated_img)

    print(f"Proses selesai! Gambar yang di crop disimpan dalam folder '{output_base}', dan gambar beranotasi disimpan sebagai '{annotated_output_path}'.")

In [ ]:
classify_apples('tes.jpg')